# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [ ]:
#The old prompt


old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [ ]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE      
create table employees(
    ID_Usr INT primary key,
    name VARCHAR);
             
    /*3 example rows
    select * from employees limit 3;
    ID_Usr    name
    1456      Amanda Nunes
    2345      Khabib Nurmagomedov
    1678      Israel Adesanya
    */

create table salary(
    ID_Usr INT,
    year DATE,
    salary FLOAT,
    foreign key (ID_Usr) references employees(ID_Usr));
             
    /*3 example rows
    select * from salary limit 3;
    ID_Usr    year          salary
    1456      01/01/2022    72000
    2345      01/01/2023    85000
    1678      01/01/2023    65000
    */

create table studies(
    ID_study INT,
    ID_Usr INT,
    educational_level INT,  /* 5=phd, 4=Master, 3=Bachelor */
    Institution VARCHAR,
    Years DATE,
    Speciality VARCHAR,
    primary key (ID_study, ID_Usr),
    foreign key(ID_Usr) references employees (ID_Usr));
             
    /*3 example rows
    select * from studies limit 3
    ID_Study ID_Usr educational_level Institution         Years       Speciality
    3451     1456   3                 University of Miami 01/01/2011  Bachelor of Mathmatics 
    4567     2345   5                 Stanford University 01/01/2019  PhD in Computer Science
    5678     1678   4                 Oxford University   01/01/2021  Master of Business Administration
    */
"""})


In [ ]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [ ]:
#new
context_user = context.copy()
print(return_CCRMSQL("""Find number of student in Mathmatics""", context_user))

```sql
SELECT COUNT(*) AS num_students
FROM studies
WHERE Speciality LIKE '%Mathematics%';
```
This query will count the number of students whose specialty includes the word "Mathematics" in the `studies` table.


In [10]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("How many student in Stanford University", old_context_user))

This is your SQL:
```sql
SELECT COUNT(*) 
FROM studies 
WHERE Institution = 'Stanford University';
```

This SQL query counts the number of students in the "studies" table who attended Stanford University by filtering the rows where the "Institution" column has the value 'Stanford University'.


In [11]:
#new
print(return_CCRMSQL("Find employees who studied at Harvard", context_user))

```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_Usr = s.ID_Usr
WHERE s.Institution = 'Harvard University';
```


In [12]:
#old
print(return_CCRMSQL("Get the average salary for each educational level. ", old_context_user))

This is your SQL:
```sql
SELECT s.educational_level, AVG(s.salary) AS average_salary
FROM salary s
JOIN studies st ON s.ID_usr = st.ID_usr
GROUP BY s.educational_level;
```
This SQL query retrieves the average salary for each educational level by joining the "salary" table with the "studies" table on the user ID. It then calculates the average salary for each educational level group using the AVG function and groups the results by educational level.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [13]:
print(return_CCRMSQL("find the highest salary among all employees", context_user))

```sql
SELECT MAX(salary) AS highest_salary
FROM salary;
```


In [14]:
print(return_CCRMSQL("find the highest salary among all employees", old_context_user))

This is your SQL:
```sql
SELECT MAX(salary) AS highest_salary
FROM salary;
```

This SQL query selects the maximum salary value from the "salary" table, giving you the highest salary among all employees.


In [15]:
print(return_CCRMSQL("find the average salary of employees who studied grouped by universoty", context_user))

```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM employees e
JOIN studies st ON e.ID_Usr = st.ID_Usr
JOIN salary sa ON e.ID_Usr = sa.ID_Usr
GROUP BY st.Institution;
```


In [16]:
print(return_CCRMSQL("find the average salary of employees who studied grouped by universoty", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS average_salary
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
GROUP BY s.Institution;
```

This SQL query joins the three tables (employees, studies, and salary) based on the employee ID. It then calculates the average salary of employees who studied at each university by grouping the results by the university.


Introduction

The Jupyter Notebook analyzed focuses on leveraging OpenAI’s GPT API to generate SQL queries dynamically based on structured table definitions. The approach includes prompt engineering, defining table structures in JSON format, and requesting SQL query generation through OpenAI’s API. The goal is to assist users in forming SQL commands efficiently.

Findings and Observations

The methodology used in the notebook was structured well, incorporating predefined context for three tables: employees, salary, and studies. The system prompt ensured GPT had a solid understanding of the table schemas, and subsequent user prompts requested SQL queries.

However, there were instances where GPT exhibited issues, such as:

Hallucinations in SQL Queries: At times, GPT introduced fields or table names that did not exist in the predefined schema. This resulted in SQL queries that referenced non-existent columns.

Inconsistent SQL Syntax: Although most queries followed SQL conventions correctly, there were cases where GPT produced incorrect syntax, particularly when handling JOIN operations or aggregations.

Ambiguity in Responses: Some SQL outputs were not concise or included unnecessary complexity, making them difficult to interpret or optimize.

Lessons Learned

Prompt Engineering is Crucial – The quality of the prompt significantly affects the SQL output. Providing detailed and structured context improves accuracy.

Verification is Necessary – GPT-generated SQL should always be reviewed before execution to ensure correctness and efficiency.

Predefining Constraints Helps – By explicitly defining column data types and table relationships, hallucinations and incorrect joins can be minimized.

Conclusion

While GPT is a powerful tool for SQL generation, its reliability depends on structured prompts and human oversight. Further refinements, such as enforcing stricter validation mechanisms or fine-tuning the prompt structure, could improve its accuracy and usefulness in real-world applications.

